# Tutorial: Learning how to use Tune

<img src="tune.png" alt="Tune Logo" width="400"/>


Tuning hyperparameters is often the most expensive part of the machine learning workflow. Tune is built to address this, demonstrating an efficient and scalable solution for this pain point.

**Code**: https://github.com/ray-project/ray/tree/master/python/ray/tune

**Examples**: https://github.com/ray-project/ray/tree/master/python/ray/tune/examples

**Documentation**: http://ray.readthedocs.io/en/latest/tune.html

**Mailing List** https://groups.google.com/forum/#!forum/ray-dev

In [ ]:
## If you are running on Google Colab, uncomment below to install the necessary dependencies 
## before beginning the exercise.

# !pip uninstall -y pyarrow
# !pip install https://s3-us-west-2.amazonaws.com/ray-wheels/latest/ray-0.8.0.dev5-cp36-cp36m-manylinux1_x86_64.whl
# !pip install ray[debug]

# # A hack to force the runtime to restart, needed to include the above dependencies.
# import os
# os._exit(0)

This tutorial will step through a couple key steps of the hyperparameter tuning process with Tune. 

1. Visualizing the data.
2. Creating a model training procedure (using Keras).
3. Tuning the model by adapting the above model training procedure to **use Tune**.
4. Analyzing the model created by Tune.

Note that this uses Tune's **function-based API**. This is mainly for prototyping. A later tutorial will cover Tune's more powerful **class-based Trainable** API.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint

from ray import tune
from ray.tune.integration.keras import TuneReporterCallback
from ray.tune.examples.utils import get_iris_data

import inspect
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

## Visualize your data

Let's first take a look at the distribution of the dataset.

### The goal of this tutorial is to have a model that can accurately predict the true label given a tuple of sepal length, petal length.

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()
true_data = iris['data']
true_label = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

def plot_data(X, y):
    # Visualize the data sets
    plt.figure(figsize=(16, 6))
    plt.subplot(1, 2, 1)
    for target, target_name in enumerate(names):
        X_plot = X[y == target]
        plt.plot(X_plot[:, 0], X_plot[:, 1], linestyle='none', marker='o', label=target_name)
    plt.xlabel(feature_names[0])
    plt.ylabel(feature_names[1])
    plt.axis('equal')
    plt.legend();

    plt.subplot(1, 2, 2)
    for target, target_name in enumerate(names):
        X_plot = X[y == target]
        plt.plot(X_plot[:, 2], X_plot[:, 3], linestyle='none', marker='o', label=target_name)
    plt.xlabel(feature_names[2])
    plt.ylabel(feature_names[3])
    plt.axis('equal')
    plt.legend();
    
plot_data(true_data, true_label)

## Creating a model training procedure (using Keras)

Now, let's define a function that will take in some hyperparameters and return a model that we can then use to train.

In [ ]:
def create_model(learning_rate, dense_1, dense_2):
    model = Sequential()
    model.add(Dense(int(dense_1), input_shape=(4,), activation='relu', name='fc1'))
    model.add(Dense(int(dense_2), activation='relu', name='fc2'))
    model.add(Dense(3, activation='softmax', name='output'))
    optimizer = SGD(lr=learning_rate)
    model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

Below is a function that trains the model using the ``create_model`` function and returns the trained model.

In [ ]:
def train_on_iris():
    train_x, train_y, test_x, test_y = get_iris_data()
    model = create_model(learning_rate=0.1, dense_1=2, dense_2=2)
    # This saves the top model
    checkpoint_callback = ModelCheckpoint("model.h5", monitor='val_loss', save_best_only=True, period=3)

    # Train the model
    model.fit(
        train_x, train_y, 
        validation_data=(test_x, test_y),
        verbose=0, batch_size=5, epochs=20, callbacks=[checkpoint_callback])
    return model

Let's quickly train the model on the dataset. The accuracy should be quite low.

In [ ]:
original_model = train_on_iris()  # This trains the model and returns it.
train_x, train_y, test_x, test_y = get_iris_data()
original_loss, original_accuracy = original_model.evaluate(test_x, test_y)
print("Loss is {:0.4f}".format(original_loss))
print("Accuracy is {:0.4f}".format(original_accuracy))

## Integrate with Tune

Now, let's use Tune to optimize a model that learns to classify Iris. This will happen in two parts - **modifying** the training function to support Tune, and then **configuring** Tune.

### Integration Part 1: Modifying the Training Function

**Instructions** Follow the next 2 steps for modifying the ``train_iris`` function to support Tune.

1. Change the signature of the function to take in a configuration dictionary.

```python
def tune_iris(config)
```
    
    
2. Pass in the configuration values into ``create_model``:

```python
model = create_model(learning_rate=config["lr"], dense_1=config["dense_1"], dense_2=config["dense_2"])
```

In [ ]:
def tune_iris():  # TODO: Change me.
    train_x, train_y, test_x, test_y = get_iris_data()
    model = create_model(learning_rate=0, dense_1=0, dense_2=0)  # TODO: Change me.
    checkpoint_callback = ModelCheckpoint("model.h5", monitor='val_loss', save_best_only=True, period=3)

    # Enable Tune to make intermediate decisions by using a Tune Callback hook. This is Keras specific.
    callbacks = [checkpoint_callback, TuneReporterCallback(freq="epoch")]
    
    # Train the model
    model.fit(
        train_x, train_y, 
        validation_data=(test_x, test_y),
        verbose=0, 
        batch_size=5, 
        epochs=20, 
        callbacks=callbacks)
    
assert len(inspect.getargspec(tune_iris).args) == 1, "The `tune_iris` function needs to take in the arg `config`."

print("Test-running to make sure this function will run correctly.")
tune.track.init()  # For testing purposes only.
tune_iris({"lr": 0.1, "dense_1": 4, "dense_2": 4})
print("Success!")

### Integration Part 2: Configuring Tune to tune hyperparameters.

**Instructions** Follow the next 2 steps to configure Tune to identify the top hyperparameters.

1. Designate the hyperparameter space. 

```python
hyperparameter_space = {
    "lr": tune.loguniform(0.0001, 0.1),  
    "dense_1": tune.uniform(2, 64),
    "dense_2": tune.uniform(2, 64),
}
```
2. Increase the number of samples. 

```python
num_samples = 20
```

#### How does parallelism work in Tune?


Setting ``num_samples`` will run a *total* of 20 trials (hyperparameter configuration samples). However, not all of them will run at once. The max training concurrency will be the number of CPU cores on the machine you're running on. For a 2-core machine, 2 models will be trained concurrently. When one is finished, a new training process will start with a new hyperparameter configuration sample. 

Each trial will run on a new Python process. The python process is killed when the trial is finished.


#### How do I debug things in Tune?

The `error file` column will show up in the output. Run the below cell with the ``error file path`` path to diagnose your issue.

```
! cat /home/ubuntu/tune_iris/tune_iris_c66e1100_2019-10-09_17-13-24x_swb9xs/error_2019-10-09_17-13-29.txt
```

In [ ]:
hyperparameter_space = {}  # TODO: Fill me out.
num_samples = 1  # TODO: Fill me out.

####################################################################################################
################ This is just a validation function for tutorial purposes only. ####################
HP_KEYS = ["lr", "dense_1", "dense_2"]
assert all(key in hyperparameter_space for key in HP_KEYS), (
    "The hyperparameter space is not fully designated. It must include all of {}".format(HP_KEYS))
######################################################################################################

analysis = tune.run(
    tune_iris, 
    verbose=1,
    config=hyperparameter_space,
    num_samples=num_samples)

assert len(analysis.trials) == 20, "Did you set the correct number of samples?"

## Analyze the best tuned model

Let's compare the real labels with the classified labels.

In [ ]:
_, _, test_data, test_labels = get_iris_data()
plot_data(test_data, test_labels.argmax(1))

In [ ]:
# Obtain the directory where the best model is saved.
logdir = analysis.get_best_logdir("keras_info/val_loss", mode="min")

# We saved the model as `model.h5` in the logdir of the trial.
from keras.models import load_model
tuned_model = load_model(logdir + "/model.h5")

tuned_loss, tuned_accuracy = tuned_model.evaluate(test_data, test_labels)
print("Loss is {:0.4f}".format(tuned_loss))
print("Tuned accuracy is {:0.4f}".format(tuned_accuracy))
print("The original un-tuned model had an accuracy of {:0.4f}".format(original_accuracy))
predicted_label = tuned_model.predict(test_data)
plot_data(test_data, predicted_label.argmax(1))

## Extra - use Tensorboard for results

You can use TensorBoard to view trial performances. If the graphs do not load, click `Toggle All Runs`.

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir ~/ray_results/tune_iris

In [ ]:
! rm -rf 